# V3 *NEW* - USE FILE TYPE & MULTI IMAGE SUPPORT W/CLAUDE


In [ ]:
# Define image in a stage and read the file
image=session.file.get_stream('@CALLAWAY_IMG_COMPLETE.PUBLIC.IMG_STAGE/ERD.jpg' , decompress=False).read() 

# Display the image
st.image(image, width=1200)

In [ ]:
/*
-- Create database if it doesn't exist
CREATE DATABASE IF NOT EXISTS CALLAWAY_IMG_COMPLETE;

-- Use the database
USE DATABASE CALLAWAY_IMG_COMPLETE;
USE SCHEMA PUBLIC;

-- Create stage for image storage
CREATE OR REPLACE STAGE CALLAWAY_IMG_COMPLETE.PUBLIC.IMG_STAGE
    FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"')
    COPY_OPTIONS = (ON_ERROR = 'CONTINUE');

-- Enhanced main table with all new fields for comprehensive golf club analysis
CREATE OR REPLACE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE_V3 (
    -- Basic identification
    IMAGE_ID VARCHAR(255) PRIMARY KEY,
    
    -- Club details (existing fields)
    CLUB_TYPE VARCHAR(50),
    CLUB_NAME VARCHAR(255),
    BRAND VARCHAR(50),
    MODEL VARCHAR(100),
    YEAR NUMBER(4,0),
    CLUB_CATEGORY VARCHAR(50),
    SHAFT_TYPE VARCHAR(50),
    SHAFT_FLEX VARCHAR(20),
    SHAFT_LABEL VARCHAR(100),                    -- Moved from enhanced
    LOFT FLOAT,
    HAND VARCHAR(10),
    
    -- *** NEW ENHANCED CLUB SPECIFICATION FIELDS ***
    CLUB_SUB_TYPE VARCHAR(50),                   -- NEW: Iron type, wedge type, etc.
    SET_COMPOSITION VARCHAR(100),                -- NEW: Set information
    LIE_ANGLE FLOAT,                            -- NEW: Lie angle in degrees
    FACE_ANGLE VARCHAR(20),                     -- NEW: Open/Closed/Neutral
    BOUNCE_ANGLE FLOAT,                         -- NEW: Bounce angle for wedges
    GRIND_TYPE VARCHAR(50),                     -- NEW: Sole grind type
    MODEL_DESIGNATION VARCHAR(100),             -- NEW: Specific model variants
    SHAFT_LENGTH_INCHES FLOAT,                  -- NEW: Shaft length
    
    -- *** NEW DETAILED CONDITION ASSESSMENT FIELDS ***
    OVERALL_GRADE NUMBER(3,1),                  -- NEW: Overall grade 1-10
    FACE_SOLE_WEAR_GRADE VARCHAR(20),          -- NEW: Face/sole wear grade
    FACE_SOLE_WEAR_DESCRIPTION VARCHAR(500),   -- NEW: Detailed wear description
    SCRATCHES_GRADE VARCHAR(20),               -- NEW: Scratch severity grade
    SCRATCHES_DESCRIPTION VARCHAR(500),        -- NEW: Scratch details
    SCRATCHES_LOCATIONS ARRAY,                 -- NEW: Array of scratch locations
    PAINT_CHIPS_GRADE VARCHAR(20),             -- NEW: Paint chip grade
    PAINT_CHIPS_DESCRIPTION VARCHAR(500),      -- NEW: Paint chip details
    PAINT_CHIPS_LOCATIONS ARRAY,              -- NEW: Array of paint chip locations
    PUTTER_PAINT_WEAR_GRADE VARCHAR(20),       -- NEW: Putter-specific paint wear
    GRIP_CONDITION VARCHAR(50),                -- NEW: Grip condition assessment
    
    -- *** NEW MARKET VALUATION FIELDS ***
    RETAIL_PRICE NUMBER(8,2),                  -- NEW: Current retail price
    TRADE_IN_VALUE NUMBER(8,2),                -- NEW: Trade-in/resale value
    MARKET_DEMAND VARCHAR(20),                 -- NEW: Market demand level
    
    -- *** NEW TECHNOLOGY AND FEATURES FIELDS ***
    TECHNOLOGY_TAGS ARRAY,                     -- NEW: Array of technology features
    SPECIAL_DESIGNATIONS ARRAY,                -- NEW: Array of special designations
    
    -- Analysis fields (existing)
    CONFIDENCE_SCORE FLOAT,
    CLASSIFICATION_NOTES VARCHAR(16777216),
    CONDITION_ASSESSMENT VARCHAR(50),
    ESTIMATED_PRICE_RANGE VARCHAR(50),
    FEATURES VARIANT,
    ANALYSIS_TIMESTAMP TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    
    -- File metadata from DIRECTORY()
    RELATIVE_PATH VARCHAR(16777216),
    FILE_URL VARCHAR(16777216),
    SIZE NUMBER,
    LAST_MODIFIED TIMESTAMP_LTZ,
    MD5 VARCHAR(32),
    
    -- Tracking fields
    CREATED_BY VARCHAR(100) DEFAULT CURRENT_USER(),
    CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    UPDATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    AI_MODEL VARCHAR(255)
);

-- *** NEW TABLE FOR MULTI-IMAGE REFERENCES V3 ***
CREATE OR REPLACE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3 (
    IMG FILE, 
    REFERENCE_ID VARCHAR(255) PRIMARY KEY DEFAULT (UUID_STRING()),
    PRIMARY_IMAGE_ID VARCHAR(255),
    REFERENCE_IMAGE_PATH VARCHAR(500),
    IMAGE_SEQUENCE NUMBER(3,0),
    REFERENCE_TYPE VARCHAR(50), -- 'MULTI_ANGLE', 'SET_MEMBER', 'COMPARISON'
    CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    FOREIGN KEY (PRIMARY_IMAGE_ID) REFERENCES CALLAWAY_CLUBS_FILE_V3(IMAGE_ID)
);

-- *** NEW TABLE FOR DETAILED DEFECT OBSERVATIONS V3 ***
CREATE OR REPLACE TABLE CALLAWAY_IMG_COMPLETE.PUBLIC.CLUB_DEFECT_OBSERVATIONS_V3 (
    OBSERVATION_ID VARCHAR(255) PRIMARY KEY,
    IMAGE_ID VARCHAR(255),  -- Changed from VARCHAR(600) to match the primary key
    DEFECT_TYPE VARCHAR(600),
    DEFECT_LOCATION VARCHAR(600),
    DEFECT_SIZE VARCHAR(600),
    DEFECT_LENGTH_MM FLOAT,
    DEFECT_WIDTH_MM FLOAT,
    DEFECT_DEPTH VARCHAR(600),
    IMPACT_ON_PERFORMANCE VARCHAR(600),
    CREATED_AT TIMESTAMP_LTZ(9) DEFAULT CURRENT_TIMESTAMP(),
    FOREIGN KEY (IMAGE_ID) REFERENCES CALLAWAY_CLUBS_FILE_V3(IMAGE_ID)
);

# 💰 Snowflake Cortex AI Cost Analysis
## 🚀 Executive Summary
> **Bottom Line:** Processing golf club images with Snowflake Cortex AI costs less than **1 cent per image!** 
---
I did 7 CORTEX.COMPLETE MULTIMODAL (see image for the count) runs today with 57 total images (also in same pic), here's the cost breakdown:
- 0.1515 Total Credits (see attached image of a streamlit app which monitors AI cost looking at SNOWFLAKE.ACCOUNT_USAGE.CORTEX_FUNCTIONS_USAGE_HISTORY)
- 0.1515 credits ÷ 7 queries = 0.0216 credits per query
- 0.0216 credits × $2.40 / credit = $0.052 per query (about 5.2 cents per query)
- 0.1515 credits × $2.40 / credit = $0.36 (36 cents total)
- Average credits per query = 0.1515 ÷ 7 = 0.0216 credits
- Cost per image = $0.36 ÷ 57 images = $0.006 (about 0.6 cents per image)

In [ ]:
USE ROLE ACCOUNTADMIN;

In [ ]:
import streamlit as st
import pandas as pd
import snowbooks_extras
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
SELECT * from CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3;

In [ ]:
SELECT COUNT(*) as TOTAL_QUERIES_TODAY
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE_V3
WHERE DATE(CREATED_AT) = CURRENT_DATE();

In [ ]:
SELECT COUNT(*) as TOTAL_IMAGE_REFERENCES_TODAY
FROM CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3
WHERE DATE(CREATED_AT) = CURRENT_DATE();

In [ ]:
SELECT * FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CLUB_DEFECT_OBSERVATIONS_V3
WHERE DATE(CREATED_AT) = CURRENT_DATE()
ORDER BY CREATED_AT DESC;

In [ ]:
SELECT * FROM CALLAWAY_IMG_COMPLETE.PUBLIC.CALLAWAY_CLUBS_FILE_V3
ORDER BY ANALYSIS_TIMESTAMP DESC;

In [ ]:
SELECT * FROM  CALLAWAY_IMG_COMPLETE.PUBLIC.IMAGE_REFERENCES_V3 
WHERE DATE(CREATED_AT) = CURRENT_DATE()
ORDER BY CREATED_AT DESC;

In [ ]:
--openai-gpt-4.1
--openai-o4-mini
--claude-4-opus
--claude-4-sonnet
--claude-3-7-sonnet
--claude-3-5-sonnet
--llama-4-maverick
--llama-4-scout
--pixtral-large
-- SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-4-sonnet',

SELECT SNOWFLAKE.CORTEX.AI_COMPLETE('openai-gpt-4.1',
PROMPT('Analyze image {0}, image {1}, image {2}, image {3}, image {4}, image {5}, image {6}, image {7} of the same golf club comprehensively and extract ALL possible information. Return ONLY valid JSON with this complete structure:
    [
    "club_type": "driver/fairway_wood/hybrid/iron/wedge/putter",
    "club_name": "full descriptive name",
    "brand": "manufacturer name", 
    "model": "specific model name",
    "year": 2023,
    "club_category": "game_improvement/players/distance/forgiveness",
    "shaft_type": "steel/graphite/hybrid",
    "shaft_flex": "extra_stiff/stiff/regular/senior/ladies",
    "shaft_label": "shaft brand and model if visible",
    "loft": 10.5,
    "hand": "right/left",
    "club_sub_type": "cavity_back/blade/mallet/etc",
    "set_composition": "individual/set_member/part_of_set",
    "lie_angle": 59.0,
    "face_angle": "neutral/open/closed",
    "bounce_angle": 12.0,
    "grind_type": "sole_grind_if_wedge",
    "model_designation": "specific_variant_or_edition",
    "shaft_length_inches": 45.0,
    "overall_grade": 8.5,
    "face_sole_wear_grade": "excellent/very_good/good/fair/poor",
    "face_sole_wear_description": "detailed description of face and sole condition including groove wear, impact marks, etc",
    "scratches_grade": "none/minor/moderate/severe",
    "scratches_description": "detailed description of all visible scratches, their location and severity",
    "scratches_locations": ["face", "sole", "crown", "back"],
    "paint_chips_grade": "none/minor/moderate/severe", 
    "paint_chips_description": "detailed description of paint/finish condition",
    "paint_chips_locations": ["crown", "sole", "back"],
    "putter_paint_wear_grade": "excellent/good/fair/poor",
    "grip_condition": "excellent/very_good/good/fair/poor/needs_replacement",
    "retail_price": 399.99,
    "trade_in_value": 150.00,
    "market_demand": "high/medium/low",
    "technology_tags": ["technology_1", "technology_2", "specific_features"],
    "special_designations": ["tour_issue", "limited_edition", "custom", "prototype"],
    "confidence_score": 0.95,
    "classification_notes": "comprehensive summary including distinguishing features, unique markings, condition assessment, technology features, and any special characteristics observed across all images",
    "condition_assessment": "excellent/very_good/good/fair/poor",
    "estimated_price_range": "$150-200",
    "features": [
        "adjustable": true,
        "forged": false,
        "cavity_back": true,
        "face_insert": false,
        "weight_ports": true,
        "alignment_aids": false,
        "special_technology": "brief_description"
    ]]
    
    Use proper JSON syntax with curly braces in your response, not square brackets. Grade condition 1-10 scale. Examine all angles for comprehensive analysis. Extract maximum data from all images provided.', 
        TO_FILE('@IMG_STAGE', 'W1.JPG'),
        TO_FILE('@IMG_STAGE', 'W2.JPG'),
        TO_FILE('@IMG_STAGE', 'W3.JPG'),
        TO_FILE('@IMG_STAGE', 'W4.JPG'),
        TO_FILE('@IMG_STAGE', 'W5.JPG'),
        TO_FILE('@IMG_STAGE', 'W6.JPG'),
        TO_FILE('@IMG_STAGE', 'W7.JPG'),
        TO_FILE('@IMG_STAGE', 'W8.JPG'))
);

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    PROMPT('Analyze image {0}, image {1}, image {2}, and image {3} of the same golf club. Identify brand, model, type, loft, and grade condition 1-10. Return JSON with club_type, brand, model, loft, overall_grade, condition_assessment, estimated_price_range.', TO_FILE('@IMG_STAGE', 'Opus_1.jpg'), 
TO_FILE('@IMG_STAGE', 'Opus_2.jpg'), 
TO_FILE('@IMG_STAGE', 'Opus_3.jpg'), 
TO_FILE('@IMG_STAGE', 'Opus_4.jpg'))
)

In [ ]:
SELECT 
    SERVICE_TYPE,
    DATE_TRUNC('MONTH', END_TIME) as MONTH,
    SUM(CREDITS_USED) as TOTAL_CREDITS,
    COUNT(DISTINCT DATE(END_TIME)) as DAYS_WITH_USAGE,
    ROUND(SUM(CREDITS_USED) / COUNT(DISTINCT DATE(END_TIME)), 2) as AVG_DAILY_CREDITS
FROM SNOWFLAKE.ACCOUNT_USAGE.METERING_HISTORY
WHERE END_TIME >= DATEADD('MONTH', -1, CURRENT_DATE)
GROUP BY 1, 2
ORDER BY 3 DESC;